In [14]:
#from Loader import fileloader as fl
#from Loader import datahandledatabase as dhdb
import ecomis
import pandas as pd
import numpy as np
import time
import posixpath
from datetime import datetime
import openpyxl as xlsx

In [15]:
periodo = 202008
#start_date_pre = '2018-08-17' # Only DDNN1
start_date = '2020-08-01' # Captura teams
end_date = '2020-08-31'# End of Captura teams

inidb = ecomis.ReadIniFile(mercado="empresas")
TESTPATH = inidb.getTestPath()
dbparser = inidb.getDbIniFileParser()

dbmanager = ecomis.DbDataProcess(str(periodo))
dbmanager.setParser(dbparser)

#start_date_pre_dt = datetime.strptime(start_date_pre, '%Y-%m-%d')
start_date_dt = datetime.strptime(start_date, '%Y-%m-%d')
end_date_dt = datetime.strptime(end_date, '%Y-%m-%d')

In [16]:
def textToDateConverter(df, date_cols):  
    for datevalue in date_cols:
        df[datevalue] = pd.to_datetime(df[datevalue],format='%Y-%m-%d', dayfirst = True)
    
    return df

In [17]:
start_time = time.time()

# PROCESO DE REPORTERIA SOBRE DESACTIVACIONES VOZ
#************************************************

COLS_DEAC = ['PERIODO_DESACTIVACION','GERENCIA1','GERENCIA2','ZONAVENTA','DEPARTAMENTO',
              'SUPERVISORKAM','VENDEDOR_ACTIVACION','VENDEDOR_OTRO','POSICION_EMPL',
              'FECHA_PROCESO','CANALDEVENTA','CODIGO','RAZON_SOCIAL','CONTRATO','FEC_ACTIV','FEC_DESACTIVA', 'TELEFONO',
              'ESTADO','PLAN_ACTIVACION','FECHA_ACTUALIZACION','PACK_CHIP','MOTIVO_DEAC','TIPODOC','DOCUMENTO',
              'CEDENTE_ACTIVACION','PORTABILIDAD',
              'TECNOLOGIAEQUIPO','ACCESS', 'ACCESSBOLSA','ACCESSPAQUETE','ACCESSLICENCIA','FECHA_CESE_EMPLEADO',
              'COMISION_UNITARIA','DIAS_DESACTIVADOS','RANGO_DESACTIVACION','CATEGORIA_MOTIVO_DEAC',
              'CATEGORIA_TECNOLOGIAEQUIPO','ACTIVIDAD', 'PENALIDAD','FACTOR_MESA_PRECIOS','REVERSION','TIPO_REVERSION']

DATE_COLUMNS = ['FECHA_PROCESO','FEC_ACTIV','FEC_DESACTIVA']
DROP_COLUMNS = ['VENDEDOR_OTRO']
RENAME_COLUMNS ={'COMISION_UNITARIA':'COMISION_UNITARIA_DB'}
#HALF_MONTHTLY_TEAMS = ['MICHA','KRATOS','KAIZEN']
HALF_MONTHTLY_TEAMS = []

CHANNEL_FILTER1 = ['VD PYMES']
CHANNEL_FILTER2 = ['CORPORACIONES','GRANDES CLIENTES','VENTA REGIONAL EMPRESA','DESARROLLO NEGOCIOS','INSIDE SALES']

ORDER_COLUMN = ['PERIODO_DESACTIVACION','GERENCIA1','GERENCIA2','DEPARTAMENTO','ZONAVENTA','SUPERVISORKAM',
                'VENDEDOR_ACTIVACION','FECHA_PROCESO','ESTADO']

deacs_ = dbmanager.loadData('View_Deacs')

deacs_ = textToDateConverter(deacs_, DATE_COLUMNS)
deacs_ = deacs_[COLS_DEAC]

#deacs_.to_csv(TESTPATH + str(periodo) + '_deacs_.csv', index = False) # Punto de control 1

filter1 = deacs_['PERIODO_DESACTIVACION'] == start_date_dt.year * 100 + start_date_dt.month
filter2 = deacs_['PERIODO_DESACTIVACION'] == end_date_dt.year * 100 + end_date_dt.month
filter3 = deacs_['ZONAVENTA'].isin(HALF_MONTHTLY_TEAMS)
filter4 = deacs_['FECHA_PROCESO'] <= end_date_dt
filter5 = deacs_['FECHA_PROCESO'] >= start_date_dt

deacs_t_1_1 = deacs_[filter1 & ~filter3]
deacs_t_1_2 = deacs_[filter1 & filter5 & filter3]
deacs_t = deacs_[filter2 & filter4 & filter3]

deacs = deacs_t_1_1.append(deacs_t_1_2, ignore_index = True)
deacs = deacs.append(deacs_t, ignore_index = True) 

#deacs.to_csv(TESTPATH + str(periodo) + '_deacs_1.csv', index = False)# Punto de control 2

stage1 = deacs['GERENCIA2'].isin(CHANNEL_FILTER2)
stage2 = deacs['GERENCIA2'].isin(CHANNEL_FILTER1)

# Filtrando las deacs_gce
#************************

deacs_gce = deacs[stage1]
# Corrigiendo los códigos de regiones múltiples
filter_reg = deacs_gce['VENDEDOR_OTRO'] != ''
#multiple_seller = deacs_gce[filter_reg]['VENDEDOR_OTRO'].values
#deacs_gce.loc[filter_reg,'VENDEDOR'] = multiple_seller # verificar si va

deacs_gce.drop(DROP_COLUMNS, axis =1, inplace = True)
deacs_gce = deacs_gce.rename(columns = RENAME_COLUMNS)
deacs_gce.sort_values(by = ORDER_COLUMN,ascending = [1,1,1,1,1,1,1,1,1], inplace = True )


#"""

# Filtrando las deacs_gcp
#************************

#"""
deacs_gcp = deacs[stage2]
deacs_gcp.drop(DROP_COLUMNS, axis =1, inplace = True)
deacs_gcp.sort_values(by = ORDER_COLUMN,ascending = [1,1,1,1,1,1,1,1,1], inplace = True )


#"""

# Exportar archivos
deacs_gce.to_csv(posixpath.join(TESTPATH, str(periodo) + '_deacs_gce.csv'), index = False, encoding='utf-8-sig')
deacs_gcp.to_csv(posixpath.join(TESTPATH, str(periodo) + '_deacs_gcp.csv'), index = False, encoding='utf-8-sig')

time_consuming = (time.time() - start_time) # In seconds

print("--- %s seconds ---" % time_consuming)

--- 15.9858717918396 seconds ---


C:\Users\jsaenza\anaconda\lib\site-packages\ipykernel_launcher.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
# Data Wrangling Deacs Data. En desarrollo
# ********************************************************
# Equipos quincenales : actualizar
start_time = time.time()

#half_monthly_teams = ['FALCO','MICHA','KRATOS','KAIZEN']
half_monthly_teams = []
end_date_dt = datetime.strptime(end_date, '%Y-%m-%d')

DATE_COLUMNS_DATA = ['ACTION_DATE']

ORDER_COLUMN_DATA = ['PERIODO_ACTIVACION','GERENCIA1','GERENCIA2','DEPARTAMENTO','ZONAVENTA','SUPERVISORKAM',
                     'VENDEDOR_CROSS_SELLING','ACTION_DATE','ESTADO']

CHANNEL_FILTER1 = ['J. FERNANDEZ','PLATAFORMAS COMERCIALES REGIONES']
CHANNEL_FILTER2 = ['CORPORACIONES','GRANDES CLIENTES','VENTA REGIONAL EMPRESA']
CHANNEL_FILTER3 = ['HOGAR']

DROP_COLUMNS = ['GERENCIA2','ZONAVENTA','VENDEDOR_OTRO']
DROP_COLUMNS2 = ['NEW_GERENCIA2','NEW_ZONAVENTA','VENDEDOR_OTRO']
RENAME_COLUMNS_DATA ={'NEW_GERENCIA2':'GERENCIA2','NEW_ZONAVENTA':'ZONAVENTA'}

COLS_DATA = ['PERIODO_ACTIVACION','GERENCIA1','GERENCIA2','NEW_GERENCIA2','ZONAVENTA','NEW_ZONAVENTA','DEPARTAMENTO',
             'SUPERVISORKAM','VENDEDOR_CROSS_SELLING','VENDEDOR_OTRO','POSICION_EMPL','ACTION_DATE','CANALDEVENTA',
             'CODIGO','RAZON_SOCIAL','CONTRATO','SERVICE','PHONENUMBER','ESTADO','FECHA_ACTUALIZACION','RUC','ACCESS_REAL',
             'TIPO2','GANADO_POR','EQUIPO','CUENTA','OBSERVACIONES']

# PROCESO DE REPORTERIA SOBRE DESACTIVACIONES DATA
#*************************************************

#deacs_ssaa = dbmanager.loadData('View_Deacs_SSAA')
#deacs_procesados.to_csv(testpath + month + '_deacs_procesados.csv', index = False)
#deacs_ssaa.to_csv(testpath + month + '_deacs_ssaa_procesados.csv', index = False)

## Revisar los contratos nulos en BD ##

time_consuming = (time.time() - start_time) # In seconds

print("--- %s seconds ---" % time_consuming)

In [ ]:
wb = xlsx.load_workbook(PATH + PERIODO + FILE)
type(wb)

In [ ]:
deacs_[deacs_['CONTRATO']==46064367]

In [5]:
deacs_

,PERIODO_DESACTIVACION,GERENCIA1,GERENCIA2,ZONAVENTA,DEPARTAMENTO,SUPERVISORKAM,VENDEDOR_ACTIVACION,POSICION_EMPL,FECHA_PROCESO,CANALDEVENTA,...,COMISION_UNITARIA,DIAS_DESACTIVADOS,RANGO_DESACTIVACION,CATEGORIA_MOTIVO_DEAC,CATEGORIA_TECNOLOGIAEQUIPO,ACTIVIDAD,PENALIDAD,REVERSION,TIPO_REVERSION,VENDEDOR_OTRO
0,202007,L. CHACON,VENTA REGIONAL EMPRESA,REGIONAL CENTRO,ICA,CAPTURA CENTRO,PMORAN,EJECUTIVO DE DESARROLLO PYME REGIONES,2020-07-24,CONSULTOR DE NEGOCIOS REGIONES,...,0.553086,179,Entre 0 y 240 dias,,,,0.0,-55.084700,REVERSION ACCESS PURO,
1,202007,L. CHACON,VENTA REGIONAL EMPRESA,REGIONAL CENTRO,ICA,CAPTURA CENTRO,PMORAN,EJECUTIVO DE DESARROLLO PYME REGIONES,2020-07-24,CONSULTOR DE NEGOCIOS REGIONES,...,0.553086,179,Entre 0 y 240 dias,,,,0.0,-55.084700,REVERSION ACCESS PURO,
2,202007,L. CHACON,VENTA REGIONAL EMPRESA,REGIONAL CENTRO,ICA,CAPTURA CENTRO,PMORAN,EJECUTIVO DE DESARROLLO PYME REGIONES,2020-07-24,CONSULTOR DE NEGOCIOS REGIONES,...,0.553086,179,Entre 0 y 240 dias,,,,0.0,-55.084700,REVERSION ACCESS PURO,
3,202007,L. CHACON,VENTA REGIONAL EMPRESA,REGIONAL CENTRO,ICA,CAPTURA CENTRO,PMORAN,EJECUTIVO DE DESARROLLO PYME REGIONES,2020-07-24,CONSULTOR DE NEGOCIOS REGIONES,...,0.553086,179,Entre 0 y 240 dias,,,,0.0,-26.694915,REVERSION ACCESS PURO,
4,202007,L. CHACON,VENTA REGIONAL EMPRESA,REGIONAL CENTRO,ICA,CAPTURA CENTRO,PMORAN,EJECUTIVO DE DESARROLLO PYME REGIONES,2020-07-24,CONSULTOR DE NEGOCIOS REGIONES,...,0.553086,179,Entre 0 y 240 dias,,,,0.0,-26.694915,REVERSION ACCESS PURO,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1425,202007,L. CHACON,VENTA REGIONAL EMPRESA,REGIONAL CENTRO,ICA,CAPTURA CENTRO,PMORAN,EJECUTIVO DE DESARROLLO PYME REGIONES,2020-07-13,CONSULTOR DE NEGOCIOS REGIONES,...,0.553086,164,Entre 0 y 240 dias,,,,0.0,-26.694916,REVERSION ACCESS PURO,
1426,202007,L. CHACON,VENTA REGIONAL EMPRESA,REGIONAL CENTRO,ICA,CAPTURA CENTRO,PMORAN,EJECUTIVO DE DESARROLLO PYME REGIONES,2020-07-13,CONSULTOR DE NEGOCIOS REGIONES,...,0.553086,164,Entre 0 y 240 dias,,,,0.0,-26.694916,REVERSION ACCESS PURO,
1427,202007,E. CACERES,GRANDES CLIENTES,ESPARTA,LIMA,,LSINCHEE,EJECUTIVO DE GRANDES CLIENTES,2020-07-27,GRANDES CLIENTES CARTERA,...,0.000000,158,Entre 0 y 240 dias,,,,0.0,-26.694910,REVERSION ACCESS PURO,
1428,202007,L. CHACON,VENTA REGIONAL EMPRESA,REGIONAL SUR,CUSCO,CAPTURA SUR,CPONCE,EJECUTIVO DE DESAROLLO PYME REGIONES,2020-07-27,CONSULTOR DE NEGOCIOS REGIONES,...,0.000000,49,Entre 0 y 240 dias,,,,0.0,-59.237288,REVERSION ACCESS PURO,
